In [ ]:
!pip install langchain

In [ ]:
!pip install gradio

In [ ]:
!pip install pypdf

In [ ]:
!pip install openai

In [ ]:
!pip install tiktoken

In [ ]:
!pip install faiss-gpu

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_qa_with_sources_chain, ConversationalRetrievalChain
import gradio as gr

loader = PyPDFLoader("/content/Nz_Info_Final.pdf")
raw_documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=20, length_function=len
)
documents = text_splitter.split_documents(raw_documents)

# Create FAISS vector store
openai_api_key = " " #replace with your own api key
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
db = FAISS.from_documents(documents, embeddings_model)
retriever = db.as_retriever()

# Initialize language model with the correct API key
llm_src = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k", openai_api_key=openai_api_key)

# Create conversational retrieval chain
retrieval_qa = ConversationalRetrievalChain.from_llm(
    llm_src,
    retriever,
    return_source_documents=True,
)

def answer_question(question):
    output = retrieval_qa({
        "question": question,
        "chat_history": []
    })
    return output['answer']

iface = gr.Interface(fn=answer_question, inputs="text", outputs="text", title="DiscoverNZ",
                     theme="light", description = 'Kia Ora! Ready to explore New Zealand?')

iface.launch(inline=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/l

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1cc61215b5e92ffa77.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
